In [19]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score, precision_score
from keras.models import Sequential
from keras.layers import Dense, Dropout

# Análise de Churn

Um arquivo de churn é um registro que acompanha e **analisa a taxa de perda** de clientes ao longo do tempo, ajudando a entender e prever o número de clientes que deixam de utilizar os serviços ou produtos da empresa.

1. OBJETIVO: PREVER a perda de clientes
2. Classe: Coluna ['Exited']


In [20]:
df = pd.read_csv('/content/drive/MyDrive/LLM/3.Apresentação de RNAs/Churn_treino.csv', sep=';')

In [21]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0,1,1,1,10134888,1
1,608,Spain,Female,41,1,8380786,1,0,1,11254258,0
2,502,France,Female,42,8,1596608,3,1,0,11393157,1
3,699,France,Female,39,1,0,2,0,0,9382663,0
4,850,Spain,Female,43,2,12551082,1,1,1,790841,0


In [22]:
df.shape

(10000, 11)

In [23]:
X = df.drop(['Exited'], axis=1) # todas as variáveis independente (exceto a classe, que é que será prevista)
y = df['Exited']

### StandardScaler

O StandardScaler transforma os dados de modo que a distribuição tenha média zero e variância unitária. Isso ajuda algoritmos de aprendizado de máquina a trabalhar melhor com dados que podem ter diferentes escalas e distribuições, tornando-os mais comparáveis e facilitando o aprendizado dos padrões nos dados.

In [24]:
StandardScaler = StandardScaler() # Instancia o StandardScaler
numerical = X.select_dtypes(include=['int64', 'float64']).columns # Seleciona as colunas numéricas do dataframe X

# Aplica o StandardScaler para padronizar as colunas numéricas selecionadas em X
X[numerical] = StandardScaler.fit_transform(X[numerical])


In [25]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,-0.326221,France,Female,0.293517,-1.041760,-1.110553,-0.911583,0.646092,0.970243,0.170614
1,-0.440036,Spain,Female,0.198164,-1.387538,0.222782,-0.911583,-1.547768,0.970243,0.353281
2,-1.536794,France,Female,0.293517,1.032908,-0.856542,2.527057,0.646092,-1.030670,0.375948
3,0.501521,France,Female,0.007457,-1.387538,-1.110553,0.807737,-1.547768,-1.030670,0.047859
4,2.063884,Spain,Female,0.388871,-1.041760,0.886252,-0.911583,0.646092,0.970243,-1.354223


## Label Encoder: transformação de dados categoricos para dados numericos

colunas: Geography	Gender

O LabelEncoder é uma técnica utilizada em machine learning para transformar variáveis categóricas (ou seja, aquelas que representam diferentes categorias, como nomes de cores ou tipos de frutas) em números inteiros. Isso é útil porque muitos algoritmos de aprendizado de máquina funcionam melhor com dados numéricos. O LabelEncoder atribui um número único para cada categoria, permitindo que os algoritmos compreendam e processem essas informações de forma mais eficaz.

In [26]:
labelEncoder = LabelEncoder() # Importar o LabelEncoder do módulo apropriado
categorical = X.select_dtypes(include=['object']).columns # Selecionar apenas colunas categóricas do dataframe X

# Iterar sobre cada coluna categórica selecionada
for col in categorical:
    X[col] = labelEncoder.fit_transform(X[col])  # Ajustar o LabelEncoder à coluna atual e transformar os valores categóricos em números inteiros


In [27]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,-0.326221,0,0,0.293517,-1.041760,-1.110553,-0.911583,0.646092,0.970243,0.170614
1,-0.440036,2,0,0.198164,-1.387538,0.222782,-0.911583,-1.547768,0.970243,0.353281
2,-1.536794,0,0,0.293517,1.032908,-0.856542,2.527057,0.646092,-1.030670,0.375948
3,0.501521,0,0,0.007457,-1.387538,-1.110553,0.807737,-1.547768,-1.030670,0.047859
4,2.063884,2,0,0.388871,-1.041760,0.886252,-0.911583,0.646092,0.970243,-1.354223


## Divisão dados em treino e teste

* 70% : treino
* 30% : teste

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

### Construção da Rede Neural do tipo SEQUENCIAL do KERAS

#### Importante
* A camada de entrada tem que ter o mesm o numero de neuronio dos nossos atributos.
* por isso que deve ser: **input_dim=X_train.shape (pois isso retorna o numero de registro dos dados)**

### model.add(Dropout(0.4))
* O dropout é uma ferramenta poderosa para regularização em redes neurais, ajudando a melhorar o desempenho e a generalização do modelo, especialmente em problemas complexos com grandes conjuntos de dados. É uma prática comum em deep learning para evitar overfitting e melhorar a capacidade de generalização dos modelos treinados.

* Dropout(0.4): Cria uma camada de dropout com uma taxa de dropout de 0.4. Isso significa que durante o treinamento, aleatoriamente, 40% das unidades (neurônios) da camada anterior serão temporariamente "desativadas" (dropadas), ou seja, suas contribuições serão ignoradas tanto na propagação para frente quanto na retropropagação de gradientes.

In [30]:
model = Sequential() # Instancia o modelo sequencial
model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1])) # Adiciona uma camada densa com 64 neurônios, função de ativação ReLU e dimensão de entrada igual ao número de colunas em X_train
model.add(Dropout(0.4)) # DROPOUT = TECNICA PARA MINIMIZAR O OVERFITTING
model.add(Dense(units=32, activation='relu')) # Adiciona outra camada densa com 32 neurônios e função de ativação ReLU
model.add(Dropout(0.4))
model.add(Dense(units=64, activation='relu')) # Adiciona outra camada densa com 64 neurônios e função de ativação ReLU
model.add(Dropout(0.4))
model.add(Dense(units=1, activation='sigmoid')) # Adiciona a camada de saída com uma unidade e função de ativação sigmóide

### TREINAMENTO DA REDE NEURAL

In [31]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # Compila o modelo com o otimizador Adam, função de perda binária de entropia cruzada e métrica de acurácia
model.fit(X_train, y_train, epochs=50, batch_size=32) # Treina o modelo com 50 épocas e tamanho de lote de 32

Epoch 1/50
219/219 [==============================] - 2s 2ms/step - loss: 0.5128 - accuracy: 0.7926
Epoch 2/50
219/219 [==============================] - 0s 2ms/step - loss: 0.4602 - accuracy: 0.7989
Epoch 3/50
219/219 [==============================] - 0s 2ms/step - loss: 0.4535 - accuracy: 0.8069
Epoch 4/50
219/219 [==============================] - 0s 2ms/step - loss: 0.4429 - accuracy: 0.8099
Epoch 5/50
219/219 [==============================] - 0s 2ms/step - loss: 0.4301 - accuracy: 0.8169
Epoch 6/50
219/219 [==============================] - 0s 2ms/step - loss: 0.4238 - accuracy: 0.8204
Epoch 7/50
219/219 [==============================] - 0s 2ms/step - loss: 0.4169 - accuracy: 0.8230
Epoch 8/50
219/219 [==============================] - 0s 2ms/step - loss: 0.4018 - accuracy: 0.8274
Epoch 9/50
219/219 [==============================] - 0s 2ms/step - loss: 0.3944 - accuracy: 0.8336
Epoch 10/50
219/219 [==============================] - 0s 2ms/step - loss: 0.3941 - accuracy: 0.8343

In [34]:
previsoes = model.predict(X_test) # Realiza previsões no conjunto de teste
previsoes

94/94 [==============================] - 0s 1ms/step


array([[0.29616532],
       [0.30661988],
       [0.18270622],
       ...,
       [0.11740954],
       [0.13046911],
       [0.6109652 ]], dtype=float32)

In [38]:
y_pred = (previsoes > 0.5).astype("int32") # Converte as previsões em valores binários (0 ou 1) 0 = FALSE e 1 = TRUE
y_pred


array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [1]], dtype=int32)

In [49]:
# Calcular as métricas
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

In [55]:
# Imprimir as métricas com uma casa decimal
print("Acurácia: {:.2f}".format(acc))
print("F1 Score: {:.2f}".format(f1))
print("Recall: {:.2f}".format(recall))
print("Precision: {:.2f}".format(precision))
print("Matriz de Confusão: ", cm)

Acurácia: 0.86
F1 Score: 0.60
Recall: 0.52
Precision: 0.71
Matriz de Confusão:  [[2247  132]
 [ 296  325]]


### Análise dos Resultados:

1. **Acurácia (Accuracy):**
   - A acurácia mede a proporção de previsões corretas feitas pelo modelo. Neste caso, o modelo alcançou uma acurácia de 86%, o que indica que 86% das previsões feitas pelo modelo estavam corretas. Isso é um bom sinal de que o modelo está geralmente correto na previsão de ambas as classes (churn e não churn).

2. **F1 Score:**
   - O F1 Score é uma média harmônica entre precisão (precision) e recall. Ele fornece uma medida única que equilibra a precisão e a completude das previsões do modelo. Um F1 Score de 0.60 sugere que o modelo está equilibrando razoavelmente bem a precisão e o recall, embora possa haver espaço para melhorias.

3. **Recall:**
   - Recall, também conhecido como taxa de verdadeiros positivos, mede a proporção de positivos reais que foram corretamente identificados pelo modelo. Um recall de 52% indica que o modelo identificou corretamente 52% dos casos de churn real. Isso sugere que há uma quantidade significativa de casos de churn que o modelo não conseguiu capturar.

4. **Precision:**
   - Precision mede a proporção de positivos previstos corretamente pelo modelo em relação ao total de positivos previstos. Com uma precisão de 71%, o modelo acerta aproximadamente 71% das previsões positivas de churn que faz. Isso indica que, das previsões feitas pelo modelo de que um cliente vai churn, aproximadamente 71% delas são corretas.

5. **Matriz de Confusão:**
   - A matriz de confusão detalha as previsões feitas pelo modelo em comparação com os resultados reais. Ela é útil para visualizar onde o modelo está acertando e onde está errando.
   - No caso apresentado:
     - **Verdadeiros Negativos (TN):** 2247 casos foram corretamente classificados como não churn.
     - **Falsos Positivos (FP):** 132 casos foram erroneamente classificados como churn.
     - **Falsos Negativos (FN):** 296 casos foram erroneamente classificados como não churn.
     - **Verdadeiros Positivos (TP):** 325 casos foram corretamente classificados como churn.

### Conclusão:

- A acurácia de 86% indica que o modelo está performando bem na classificação geral, mas é importante observar que o recall e o F1 Score estão relativamente mais baixos. Isso sugere que o modelo pode estar perdendo alguns casos de churn (baixo recall) enquanto mantém uma boa precisão geral (71%).
- Para melhorar o desempenho, seria interessante explorar ajustes adicionais no modelo, como otimização de hiperparâmetros, seleção de características mais relevantes ou até mesmo considerar o uso de outras técnicas como balanceamento de classes para melhorar a detecção de churn (aumentando o recall).